In [1]:
import pandas as pd
import numpy as np
import copy
import sys
import os
sys.path.append("../")
from parser.utils import load_json, dfs_cardinality, estimate_scan_in_mb
from models.feature.single_xgboost_feature import find_top_k_operators, featurize_one_plan, get_top_k_table_by_size
from utils.load_brad_trace import load_trace, create_concurrency_dataset, load_trace_all_version
from models.concurrency.utils import pre_info_train_test_seperation
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from models.single.stage import SingleStage
from models.concurrency.complex_models import ConcurrentRNN
np.set_printoptions(suppress=True)

In [2]:
parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_aurora/aurora_mixed_parsed_queries.json"
plans = load_json(parsed_queries_path, namespace=False)
folder_name = "mixed_postgres"
directory = f"/Users/ziniuw/Desktop/research/Data/AWS_trace/{folder_name}/"
all_raw_trace, all_trace = load_trace_all_version(directory, 8, concat=True)
all_concurrency_df = []
for trace in all_trace:
    concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=200)
    all_concurrency_df.append(concurrency_df)
concurrency_df = pd.concat(all_concurrency_df, ignore_index=True)

In [3]:
train_trace_df_sep, eval_trace_df_sep = pre_info_train_test_seperation(concurrency_df)
print(len(train_trace_df_sep), len(eval_trace_df_sep))
np.random.seed(0)
train_idx = np.random.choice(len(concurrency_df), size=int(0.8 * len(concurrency_df)), replace=False)
test_idx = [i for i in range(len(concurrency_df)) if i not in train_idx]
train_trace_df = copy.deepcopy(concurrency_df.iloc[train_idx])
eval_trace_df = concurrency_df.iloc[test_idx]
eval_trace_df = copy.deepcopy(eval_trace_df[eval_trace_df['num_concurrent_queries'] > 0])
print(len(train_trace_df), len(eval_trace_df))

17603 13399
24889 6198


In [4]:
ss = SingleStage(use_table_features=True, true_card=True)
#df = ss.featurize_data(train_trace_df, parsed_queries_path)
df = ss.featurize_data(concurrency_df, parsed_queries_path)
ss.train(df)

Top 20 operators contains 0.9650782102582758 total operators


In [6]:
rnn = ConcurrentRNN(ss, 
                    "postgres",
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=True,
                    use_separation=False
                   )
rnn.train(train_trace_df, eval_trace_df, lr=0.001, loss_function="q_loss", val_on_test=True)
#rnn.load_model("checkpoints")

********Epoch 0, training loss: 347.58929279981515 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 17.19it/s]


50% absolute error is 4.008946895599365, q-error is 1.669534981250763
90% absolute error is 56.427736282348675, q-error is 5.132400846481324
95% absolute error is 99.74682159423816, q-error is 7.393720984458918
********Epoch 5, training loss: 154.66684285944854 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.91it/s]


50% absolute error is 3.3501548767089844, q-error is 1.394617736339569
90% absolute error is 21.596798706054688, q-error is 3.6809489250183107
95% absolute error is 35.74762611389158, q-error is 6.075449180603016
********Epoch 10, training loss: 0.27595379108037704 || evaluation loss: ********


...

100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.65it/s]


50% absolute error is 2.5001492500305176, q-error is 1.2890622019767761
90% absolute error is 17.25225067138672, q-error is 2.8979191064834593
95% absolute error is 26.077169609069774, q-error is 4.194843029975887
********Epoch 120, training loss: 4.625079183127635 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.67it/s]


50% absolute error is 2.648329734802246, q-error is 1.2909478545188904
90% absolute error is 18.16018352508545, q-error is 2.822620964050293
95% absolute error is 26.905928230285642, q-error is 4.047429370880127


KeyboardInterrupt: 

In [9]:
rnn.save_model("checkpoints")

In [6]:
rnn2 = ConcurrentRNN(ss, 
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=False,
                    use_seperation=False
                   )
rnn2.train(train_trace_df_sep, eval_trace_df_sep, lr=0.001, loss_function="q_loss", val_on_test=True)

********Epoch 0, training loss: 401.60364659328377 || evaluation loss: ********


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 132.50it/s]


50% absolute error is 4.02608060836792, q-error is 3.915198802947998
90% absolute error is 20.422407150268555, q-error is 31.662242889404297
95% absolute error is 34.7833251953125, q-error is 59.1192626953125
********Epoch 5, training loss: 0.3782997516404211 || evaluation loss: ********


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 119.85it/s]


50% absolute error is 2.1188182830810547, q-error is 2.816147804260254
90% absolute error is 11.656293869018555, q-error is 18.462446212768555
95% absolute error is 25.714630126953125, q-error is 33.27926254272461
********Epoch 10, training loss: 0.33071399506478183 || evaluation loss: ********


...

50% absolute error is 2.1996700763702393, q-error is 2.830948829650879
90% absolute error is 9.968746185302734, q-error is 17.36515235900879
95% absolute error is 25.595638275146484, q-error is 30.142972946166992
********Epoch 190, training loss: 7.856241999375346 || evaluation loss: ********


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 117.27it/s]


50% absolute error is 2.0888543128967285, q-error is 2.7173123359680176
90% absolute error is 10.123455047607422, q-error is 16.875577926635742
95% absolute error is 25.55385971069336, q-error is 30.01237678527832
********Epoch 195, training loss: 7.147099245998976 || evaluation loss: ********


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 113.78it/s]


50% absolute error is 2.354909658432007, q-error is 2.9794392585754395
90% absolute error is 10.141586303710938, q-error is 19.38707733154297
95% absolute error is 25.74962043762207, q-error is 32.55106735229492


In [8]:
preds, labels = rnn.predict(eval_trace_df, use_pre_info_only=False)

100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:03<00:00, 16.06it/s]

50% absolute error is 2.4635677337646484, q-error is 1.2799019813537598
90% absolute error is 17.452255630493166, q-error is 2.8439884185791016
95% absolute error is 26.966872787475545, q-error is 4.003956580162048


In [44]:
preds2, labels2 = rnn2.predict(eval_trace_df_sep, use_pre_info_only=False)

100%|████████████████████████████████████████| 200/200 [00:01<00:00, 139.35it/s]


50% absolute error is 3.380521774291992, q-error is 4.0659284591674805
90% absolute error is 40.663795471191406, q-error is 47.04194641113281
95% absolute error is 59.644752502441406, q-error is 101.35169982910156


In [48]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds2[i])[idx], np.asarray(preds[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

111


array([[  2.8291447,   7.1109896,  79.540054 ],
       [  2.8653407,   2.3412442,   3.416112 ],
       [  2.934177 ,   4.0760264,   4.0271263],
       [  2.9910045,   3.7841144,   3.8717294],
       [  3.0511217,   5.0034904,   3.1730235],
       [  3.148857 ,  18.968218 ,   3.1449356],
       [  3.170916 ,   6.0808353, 106.32712  ],
       [  3.183551 ,   6.5285897,   4.7487164],
       [  3.2096725,  33.221123 ,   2.8220966],
       [  3.2253332,   5.26631  ,   3.3115742],
       [  3.246746 ,   4.4229527,   5.8701944],
       [  3.2892797,   5.2571554,   7.374345 ],
       [  3.2927618,  24.601357 ,   2.9005537],
       [  3.3001895,   3.752159 ,   4.9782295],
       [  3.3236983,   3.4375095,   4.804597 ],
       [  3.3453178,   4.3794765,   3.0326128],
       [  3.4195132,   5.3936315,   8.1688   ],
       [  3.4844189,   6.0924673,   4.3422008],
       [  3.5340056,   4.0386086,   4.554105 ],
       [  3.5757537,   8.585522 ,  32.302185 ],
       [  3.5846162,   6.08425  ,   5.50

In [8]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(preds2[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

NameError: name 'preds2' is not defined

In [11]:
i = 2
idx = np.argsort(preds[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(labels[i])[idx]), axis=1)

7


array([[  9.086283,  10.558472],
       [  9.326768,  10.257236],
       [ 16.062849,  39.89623 ],
       [ 20.419281,  69.41913 ],
       [ 35.720085,  52.07332 ],
       [ 44.310814, 110.97227 ],
       [ 50.8906  ,  45.67449 ]], dtype=float32)